# Create dummies

In [157]:
import pandas as pd
import numpy as np
import sys 
import csv
import unicodedata
import collections
import scipy

In [158]:
df = pd.read_csv('data_19_noGoalkeeper.csv', encoding = "utf8")

In [159]:
df.shape

(767, 144)

In [160]:
# more than 1 transfer in 3 years
tot = [item for item, count in collections.Counter(df.name_final).items() if count > 1]
df['more1t'] = np.nan
df['more1t'] = [1 if x in tot else 0 for x in df['name_final']]

In [161]:
# dummy more than 18 years
df['d18'] = np.nan
df['d18'] = [1 if x <19 else 0 for x in df['Age']]

In [162]:
df.shape

(767, 146)

In [163]:
# liquidity
df1 = df.groupby(["wintersummer", "year"]).size().reset_index(name="liq")
df2 = pd.merge(df, df1,  how='outer', left_on=['wintersummer','year'], right_on = ['wintersummer','year'])

In [164]:
df2.shape

(767, 147)

In [165]:
# national performance == 0 devo dividere B-C-D
bcd = pd.read_csv('bcd.csv', encoding='utf8').drop(columns=['weight','height','Unnamed: 0','dummyA'], axis=1)

In [166]:
bcd.head()

,date,season,team_from,div
0,02/07/2017,17/18,1860 Munchen,c
1,02/07/2017,17/18,Almeria,b
2,27/07/2018,18/19,Almeria,b
3,30/01/2017,16/17,Ascoli,b
4,15/06/2018,18/19,Ascoli,b


In [167]:
pd.to_datetime(df2.date)
date_format = '%Y-%m-%d'
df2['date'] = pd.to_datetime(df2['date'])
df2.date

pd.to_datetime(bcd.date)
date_format = '%d/%m/%Y'
bcd['date'] = pd.to_datetime(bcd['date'])

In [168]:
df3 = pd.merge(df2, bcd,  how='outer', left_on=['team_from','date'], right_on = ['team_from','date'])

df3['div'] = df3['div'].fillna('a')
df3['dA'] = np.nan
df3['dB'] = np.nan
df3['dC'] = np.nan

df3['dA'] = [1 if x == 'a' else 0 for x in df3['div']]
df3['dB'] = [1 if x == 'b' else 0 for x in df3['div']]
df3['dC'] = [1 if x == 'c' else 0 for x in df3['div']]

In [169]:
df3.shape

(777, 152)

In [170]:
df3 = df3.drop_duplicates() # non so perché si creano ma li tolgo

In [172]:
df3.shape

(767, 152)

### rimuovo ultime righe che sono un errore che viene dal merge

In [173]:
# Days from open of the transfer window of the buyer team -> 0 if negative

df3.days_from_openBuying[df3.days_from_openBuying < 0] = 0

# Drop columns

In [174]:
df3 = df3.drop(columns=['Unnamed: 0','year','birthDate','nationalPerf_team_from','shortName','dummyAB'], axis=1)

df4 = df3.drop(columns=['country_name_selling','name_final','country_name_buying','date','city_selling','city_buying','PassportCountry'], axis=1)

In [175]:
df3.to_csv('data_graphs.csv', header=True, sep=',')

# Rename columns

In [176]:
df4.rename(columns={'team_from':'TF','team_to':'TT','wintersummer':'ws', 'country_selling':'countryS','country_buying':'countryB','birthCountry':'birth','europeanPerf_team_from':'euroTS','europeanPerf_team_to':'euroTB','dummyfirst5':'d5','dummyfirst10':'d10','dummyfirst20':'d20','dummySheikh':'dSheikh','days_from_openBuying':'openB','europeanPerf_country_from':'euroCS','europeanPerf_country_to':'euroCB','number_export_players':'noExp','number_import_players':'noImp','abs_number_per_nation_sold':'noCS','abs_number_per_nation_bought':'noCB','number_matches':'noMatch','number_matches1Year':'noMatch1Y','number_matches6Months':'noMatch6','perc_s_actions':'actionS','goal_per_match':'goals','assist_per_match':'assist','perc_shots_target':'shotS','passes_acc':'passA','lpasses_acc':'lpassA','crosses_acc':'crossA','perc_dribbles':'dribbS','perc_duels':'duelS','perc_aerialduels':'aerDuelS','interceptions':'interc','rec_opposite':'recOpp','rec_tot':'recT','los_own':'losOwn','los_tot':'losTot','minutes1Year':'min1Y','perc_s_actions1Year':'actionS1Y','goal_per_match1Year':'goal1Y','assist_per_match1Year':'assist1Y','perc_shots_target1Year':'shotS1Y','xg1Year':'xg1Y','passes_acc1Year':'passA1Y','lpasses_acc1Year':'lpassA1Y','crosses_acc1Year':'crossA1Y','perc_dribbles1Year':'dribbS1Y','perc_duels1Year':'duelS1Y','perc_aerialduels1Year':'aerDuelS1Y','interceptions1Year':'interc1Y','rec_opposite1Year':'recOpp1Y','rec_tot1Year':'recT1Y','los_own1Year':'losOwn1Y','los_tot1Year':'losT1Y','yellow1Year':'yell1Y','red1Year':'red1Y','minutes1Year':'min1Y','minutes6Months':'min6','perc_s_actions6Months':'actionS6','goal_per_match6Months':'goal6','assist_per_match6Months':'assist6','perc_shots_target6Months':'shotS6','xg6Months':'xg6','passes_acc6Months':'passA6','lpasses_acc6Months':'lpassA6','crosses_acc6Months':'crossA6','perc_dribbles6Months':'dribbS6','perc_duels6Months':'duelS6','perc_aerialduels6Months':'aerDuelS6','interceptions6Months':'interc6','rec_opposite6Months':'recOpp6','rec_tot6Months':'recT6','los_own6Months':'losOwn6','los_tot6Months':'losT6','yellow6Months':'yell6','red6Months':'red6','shots_per_match':'shotT','perc_shots_target_team':'shotST','xg_team':'xgT','goals_per_match_team':'goalT','passes_per_match':'passT','passes_accuracy':'passAT','perc_possession_team':'possT','los_per_match_team':'losT','low_losses_team':'lowLosT','rec_per_match_team':'recT','high_recoveries_team':'highRecT','challenges_per_match_team':'challT','perc_challenges_won_team':'challST','total_goals_opponents':'goalOppT','total_goals':'goalAbsT','shots_per_match1Year':'shotsT1Y','perc_shots_target_team1Year':'shotST1Y','xg_team1Year':'xgT1Y','goals_per_match_team1Year':'goalT1Y','passes_per_match1Year':'passT1Y','passes_accuracy1Year':'passAT1Y','perc_possession_team1Year':'possT1Y','los_per_match_team1Year':'losT1Y','low_losses_team1Year':'lowLosT1Y','rec_per_match_team1Year':'recT1Y','high_recoveries_team1Year':'highRecT1Y','challenges_per_match_team1Year':'challT1Y','perc_challenges_won_team1Year':'challST1Y','total_goals_opponents1Year':'goalOppT1Y','total_goals1Year':'goalAbsT1Y','shots_per_match6Months':'shotT6','perc_shots_target_team6Months':'shotST6','xg_team6Months':'xgT6','goals_per_match_team6Months':'goalT6','passes_per_match6Months':'passT6','passes_accuracy6Months':'passAT6','perc_possession_team6Months':'possT6','los_per_match_team6Months':'losT6','low_losses_team6Months':'lowLosT6','rec_per_match_team6Months':'recT6','high_recoveries_team6Months':'highRecT6','challenges_per_match_team6Months':'challT6','perc_challenges_won_team6Months':'challST6','total_goals_opponents6Months':'goalOppT6','total_goals6Months':'goalAbsT6'}, 
                 inplace=True)

In [177]:
pd.set_option('display.max_columns', 500)
df3.head()

,date,season_x,height,weight,team_from,team_to,price,wintersummer,role,country_selling,country_name_selling,city_selling,country_buying,country_name_buying,city_buying,PassportCountry,birthCountry,foot,name_final,europeanPerf_team_from,europeanPerf_team_to,dummyfirst5,dummyfirst10,dummyfirst20,dummySheikh,Age,days_from_openBuying,europeanPerf_country_from,europeanPerf_country_to,number_export_players,number_import_players,abs_number_per_nation_sold,abs_number_per_nation_bought,number_matches,minutes,perc_s_actions,goal_per_match,assist_per_match,perc_shots_target,xg,passes_acc,lpasses_acc,crosses_acc,perc_dribbles,perc_duels,perc_aerialduels,interceptions,rec_opposite,rec_tot,los_own,los_tot,yellow,red,number_matches1Year,minutes1Year,perc_s_actions1Year,goal_per_match1Year,assist_per_match1Year,perc_shots_target1Year,xg1Year,passes_acc1Year,lpasses_acc1Year,crosses_acc1Year,perc_dribbles1Year,perc_duels1Year,perc_aerialduels1Year,interceptions1Year,rec_opposite1Year,rec_tot1Year,los_own1Year,los_tot1Year,yellow1Year,red1Year,number_matches6Months,minutes6Months,perc_s_actions6Months,goal_per_match6Months,assist_per_match6Months,perc_shots_target6Months,xg6Months,passes_acc6Months,lpasses_acc6Months,crosses_acc6Months,perc_dribbles6Months,perc_duels6Months,perc_aerialduels6Months,interceptions6Months,rec_opposite6Months,rec_tot6Months,los_own6Months,los_tot6Months,yellow6Months,red6Months,shots_per_match,perc_shots_target_team,xg_team,goals_per_match_team,passes_per_match,passes_accuracy,perc_possession_team,los_per_match_team,low_losses_team,rec_per_match_team,high_recoveries_team,challenges_per_match_team,perc_challenges_won_team,total_goals_opponents,total_goals,shots_per_match6Months,perc_shots_target_team6Months,xg_team6Months,goals_per_match_team6Months,passes_per_match6Months,passes_accuracy6Months,perc_possession_team6Months,los_per_match_team6Months,low_losses_team6Months,rec_per_match_team6Months,high_recoveries_team6Months,challenges_per_match_team6Months,perc_challenges_won_team6Months,total_goals_opponents6Months,total_goals6Months,shots_per_match1Year,perc_shots_target_team1Year,xg_team1Year,goals_per_match_team1Year,passes_per_match1Year,passes_accuracy1Year,perc_possession_team1Year,los_per_match_team1Year,low_losses_team1Year,rec_per_match_team1Year,high_recoveries_team1Year,challenges_per_match_team1Year,perc_challenges_won_team1Year,total_goals_opponents1Year,total_goals1Year,more1t,d18,liq,season_y,div,dA,dB,dC
0,2016-08-31,16/17,188,81,Fiorentina,Chelsea,27.0,s,Defender,IT,Italy,Firenze,GB,England,London,Spain,Spain,left,Marcos Alonso,8.0,18.0,0,1,1,0,26,83,11.500,14.25,9,15,192,224,44,89,63.77,0.068,0.091,26.15,0.104,83.37,53.73,26.57,76.65,41.96,57.67,5.18,2.11,6.95,4.07,10.09,9,0,38,89,64.44,0.026,0.105,24.56,0.096,83.40,55.00,26.32,78.95,42.66,55.95,5.00,2.21,6.84,3.82,9.92,7,0,13,90,63.66,0.077,0.077,20.00,0.120,82.30,56.16,23.91,76.19,42.67,57.41,5.77,1.85,8.00,3.62,9.85,2,0,13.81,32.87,1.205,1.519,540.48,87.28,58.20,98.90,0.18,78.42,0.14,245.98,39.24,60,79,14.46,30.32,1.182,1.154,536.00,86.73,58.29,101.15,0.17,79.38,0.15,247.15,38.38,18,15,14.17,32.52,1.217,1.543,552.96,87.35,59.44,99.98,0.18,79.91,0.14,250.87,39.45,52,71,0,0,199,NaN,a,1,0,0
1,2016-08-31,16/17,179,75,Lorient,Sunderland,16.0,s,Midfielder,FR,France,Lorient,GB,England,Sunderland,Gabon,Gabon,right,D. Ndong,0.0,0.0,0,0,0,0,22,83,11.083,14.25,3,2,99,224,35,90,66.27,0.057,0.057,20.59,0.051,90.68,70.93,35.48,65.81,35.49,37.16,4.54,3.43,9.23,3.97,8.77,12,1,35,90,66.27,0.057,0.057,20.59,0.051,90.68,70.93,35.48,65.81,35.49,37.16,4.54,3.43,9.23,3.97,8.77,12,1,12,90,67.78,0.000,0.000,28.57,0.005,90.80,67.86,27.27,67.65,35.46,26.92,4.42,2.33,8.25,4.17,8.25,2,1,11.12,39.04,1.135,1.341,443.32,84.00,48.87,107.63,0.16,83.59,0.11,257.76,37.97,67,55,9.57,36.57,0.891,0.643,428.14,83.80,47.40,107.43,0.17,81.29,0.11,244.50,37.06,27,9,11.12,39.04,1.135,1.341,443.32,84.00,48.87,107.63,0.16,83.59,0.11,257.76,37.97,67,55,0,0,199,NaN,a,1,0,0
2,2016-08-31,16/17

In [178]:
nomicol = pd.DataFrame(df4.columns)

In [179]:
df4.to_csv('data_models.csv', header=True, sep=',')

In [180]:
df4.shape

(767, 139)

In [186]:
sum(df4.noCB.unique())-767

57